In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import requests
import re
from bs4 import BeautifulSoup

In [2]:
path = os.path.join('.','csv', 'webarchive')


In [3]:
path

'.\\csv\\webarchive'

In [24]:
urls = [
    'https://web.archive.org/web/20200229172202/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    "https://web.archive.org/web/20200301211249/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html",
    'https://web.archive.org/web/20200302144925/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200303190132/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200304170930/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200305224400/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200306191524/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200307215000/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200308234917/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200309221216/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200310235804/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200311231057/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html',
    'https://web.archive.org/web/20200312233210/https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html'
    ]
df_new = pd.DataFrame()
for url in urls:
    archive_html = requests.get(url).text
    soup = BeautifulSoup(archive_html, 'html.parser')
    try:
        stand = soup.find( text=re.compile(".*[Ss]tand:.*?(\d+\.\d+\.2020).*"))
    except:
        print('Fehler')
    print(stand)
    stand = re.search("[Ss]tand.*?(\d+\.\d+\.2020)", str(stand)).group(1)
    print(stand)
    main = soup.find('div', {'id':'main'})
    archive_table = main.find('table')
    df_wa = pd.read_html(str(archive_table))[0]
    pd_date = pd.to_datetime(stand,format='%d.%m.%Y')
    df_wa['date'] = pd_date
    if (pd_date <= pd.to_datetime('2020/03/05')):
        df_wa.columns = ['Bundesland', 'confirmed', 'date']
    elif (pd_date <= pd.to_datetime('2020/03/08')):
        df_wa.columns =['Bundesland', 'confirmed', 'extra','date']
        df_wa.confirmed = df_wa.confirmed.astype(int)
    elif (pd_date <= pd.to_datetime('2020/03/11')):
        df_wa.columns =['Bundesland', 'confirmed', 'deaths', 'extra','date']
        df_wa.confirmed = df_wa.confirmed.astype(int)
    else:
        df_wa.columns = ['Bundesland', 'confirmed', 'confirmed_el', 'extra', 'date']
    #print(df_wa)
    df_new = pd.concat([df_wa, df_new])

df_new['deaths'] = df_new.confirmed.str.extract(r'\((\d+)\)')
df_new['confirmed'] = df_new.confirmed.replace(r'(\d+) \(\d+\)', r'\1', regex=True)
df_new.deaths.replace({np.nan:0}, inplace=True)

Stand: 29.2.2020
29.2.2020
Stand: 1.3.2020
1.3.2020
Stand: 2.3.2020
2.3.2020
Stand: 3.3.2020
3.3.2020
(Datenstand: 4.3.2020, 15:00 Uhr)
4.3.2020
(Datenstand: 5.3.2020, 15:00 Uhr)
5.3.2020
(Datenstand: 6.3.2020, 15:00 Uhr)
6.3.2020
(Stand: 7.3.2020, 15:00 Uhr)
7.3.2020
(Stand: 8.3.2020, 15:00 Uhr)
8.3.2020
(Stand: 9.3.2020, 15:00 Uhr)
9.3.2020
Stand: 10.3.2020, 15:00 Uhr (online aktualisiert um 21:30 Uhr)
10.3.2020
Stand: 11.3.2020, 15:00 Uhr (online aktualisiert um 18:10 Uhr)
11.3.2020

Stand: 12.03.2020 15:00 Uhr (online aktualisiert um 19:30 Uhr)
12.03.2020


In [27]:
df_new

,Bundesland,confirmed,confirmed_el,extra,date,deaths
0,Baden-Württemberg,454,331 (1),NaN,2020-03-12,1
1,Bayern,500,317,NaN,2020-03-12,1
2,Berlin,137,137,NaN,2020-03-12,0
3,Brandenburg,30,23,NaN,2020-03-12,0
4,Bremen,38,35,NaN,2020-03-12,0
...,...,...,...,...,...,...
3,Nordrhein-Westfalen,30,NaN,NaN,2020-02-29,0
4,Rheinland-Pfalz,1,NaN,NaN,2020-02-29,0
5,Schleswig Holstein,1,NaN,NaN,2020-02-29,0
6,Repatriierte,2,NaN,NaN,2020-02-29,0
